In [1]:
import pandas as pd

In [2]:
data = pd.read_stata('../02. Datasets/ttest_with_result_mean_std.dta')
data.columns = data.iloc[0]
# Drop the first row
data = data.drop(0).reset_index(drop=True)

In [3]:
for i in range(1, 12):
    data.loc[i, 'SD'] = data.loc[i, 'Mean']
    data.loc[i, 'Mean'] = data.loc[i, 'Variable']
    data.loc[i, 'Variable'] = data.loc[i, 'abc']
    data.loc[i, 'abc'] = 0

    data.loc[12 + i, 'SD'] = data.loc[12 + i, 'Mean']
    data.loc[12 + i, 'Mean'] = data.loc[12 + i, 'Variable']
    data.loc[12 + i, 'Variable'] = data.loc[12 + i, 'abc']
    data.loc[12 + i, 'abc'] = 1

now we merge the table with the given label

In [4]:
df = pd.read_excel('../02. Datasets/variable__label_correspondence.xlsx')
df['varlab'] = df['varlab'].str.replace(r'^s\d+q\d+_\d+\s+', '', regex=True)
df['varlab'] = df['varlab'].str.replace(r's22q1_15a ', '') # s22q1_15a 
df['varlab'] = df['varlab'].str.replace(r'id11 ', '')

In [5]:
data['Variable'].replace('edchild pe~t', 'edchild_percent', inplace  = True)
data['Variable'].replace('accesscell~e', 'accesscellphone', inplace  = True)
data['Variable'].replace('eth hausa	', 'edchild_percent', inplace  = True)
data['Variable'].replace('accesscell~e', 'accesscellphone', inplace  = True)


In [6]:
data['Variable'] = data['Variable'] \
    .str.replace(' ', '_') \
    .str.replace('edchild_pe~t', 'edchild_percent') \
    .str.replace('eth_hausa', 'eth_hausa') \
    .str.replace('hhmem_no', 'hhmem_no') \
    .str.replace('accesscell~e', 'accesscellphone')


In [7]:
data = data.merge(df, left_on='Variable', right_on='name', how='left')

In [8]:
dataframe = data[['Variable', 'varlab', 'Mean', 'SD', 'abc']]

In [9]:
df_abc_0 = dataframe[dataframe['abc'] == 0].reset_index(drop=True)
df_abc_1 = dataframe[dataframe['abc'] == 1][['varlab', 'Mean', 'SD']].reset_index(drop=True)

df_abc_1.rename(columns={'Mean': 'Mean with abc', 'SD': 'SD with abc'}, inplace=True)

result_df = pd.merge(df_abc_0, df_abc_1, on='varlab', how='left')
result_df.rename(columns={'Mean': 'Mean without abc', 'SD': 'SD without abc'}, inplace=True)

In [10]:
result_df.drop(columns = ['abc'], inplace = True)

In [11]:
result_df

,Variable,varlab,Mean without abc,SD without abc,Mean with abc,SD with abc
0,hhhead,Are you the household head?,0.560,0.497,0.547,0.498
1,eth_hausa,Respondent is Hausa,0.715,0.452,0.721,0.449
2,hhmem_no,Number of household members,8.422,4.054,8.328,4.074
3,edchild_percent,Percentage of children under 15 who have some ...,0.279,0.276,0.269,0.270
4,assets,Number of asset categories owned by household,4.990,1.609,4.979,1.575
5,drought,Household experienced drought in past year,0.385,0.487,0.380,0.486
6,cellphone,Household owns a cell phone (excluding group p...,0.296,0.457,0.295,0.457
7,accesscellphone,Access to household or village-level cell phone,0.763,0.426,0.798,0.402
8,usecellphone,Respondent has used cell phone since last harvest,0.542,0.499,0.573,0.495
9,makecall,Respondent has made call,0.691,0.463,0.725,0.447


In [12]:
difference = pd.read_stata('../02. Datasets/Table1_PanelA_dta.dta')
difference.columns = difference.iloc[1]
# Drop the first row
difference = difference.drop(0).reset_index(drop=True)
difference = difference.drop(0).reset_index(drop=True)

result_data = pd.DataFrame(difference.iloc[1])
result_data['std'] = difference.iloc[2]
result_data.columns = result_data.iloc[0]

result_data.drop('VARIABLES', inplace=True)

result_data.reset_index(drop = False, inplace = True)
result_data.reset_index(drop = True, inplace = True)

In [13]:
result_data.rename(columns = {1: 'Name', 'abc': 'Diff', '': 'std'}, inplace = True)

In [14]:
result_data.reset_index(drop = True, inplace = True)

In [15]:
result_df = pd.merge(result_df, result_data, left_on= 'Variable', right_on='Name', how='left')

In [16]:
result_df.drop(columns = ['Variable', 'Name'], inplace = True)
result_df.rename(columns = {'varlab': 'Variable'}, inplace = True)

In [31]:
latex_table  = result_df.to_latex(index=False, escape=False)

In [32]:
print(latex_table)

\begin{tabular}{lllllll}
\toprule
Variable & Mean without abc & SD without abc & Mean with abc & SD with abc & Diff & std \\
\midrule
Are you the household head? & 0.560 & 0.497 & 0.547 & 0.498 & -0.01 & (0.02) \\
Respondent is Hausa & 0.715 & 0.452 & 0.721 & 0.449 & 0.01 & (0.03) \\
Number of household members & 8.422 & 4.054 & 8.328 & 4.074 & 0.02 & (0.25) \\
Percentage of children under 15 who have some education & 0.279 & 0.276 & 0.269 & 0.270 & -0.00 & (0.02) \\
Number of asset categories owned by household & 4.990 & 1.609 & 4.979 & 1.575 & -0.03 & (0.10) \\
Household experienced drought in past year & 0.385 & 0.487 & 0.380 & 0.486 & -0.03 & (0.03) \\
Household owns a cell phone (excluding group phone) & 0.296 & 0.457 & 0.295 & 0.457 & -0.00 & (0.03) \\
Access to household or village-level cell phone & 0.763 & 0.426 & 0.798 & 0.402 & 0.04* & (0.02) \\
Respondent has used cell phone since last harvest & 0.542 & 0.499 & 0.573 & 0.495 & 0.04 & (0.03) \\
Respondent has made call & 0.6

In [36]:
parts = latex_table.split("\\midrule", 1)
new_line = "\\multicolumn{7}{l}{\\textbf{Panel A: pre-program household characteristics}}  \\\\"
modified_table = parts[0] + "\\midrule\n" + new_line + parts[1]


In [37]:
print(modified_table)

\begin{tabular}{lllllll}
\toprule
Variable & Mean without abc & SD without abc & Mean with abc & SD with abc & Diff & std \\
\midrule
\multicolumn{7}{l}{\textbf{Panel A: pre-program household characteristics}}  \\
Are you the household head? & 0.560 & 0.497 & 0.547 & 0.498 & -0.01 & (0.02) \\
Respondent is Hausa & 0.715 & 0.452 & 0.721 & 0.449 & 0.01 & (0.03) \\
Number of household members & 8.422 & 4.054 & 8.328 & 4.074 & 0.02 & (0.25) \\
Percentage of children under 15 who have some education & 0.279 & 0.276 & 0.269 & 0.270 & -0.00 & (0.02) \\
Number of asset categories owned by household & 4.990 & 1.609 & 4.979 & 1.575 & -0.03 & (0.10) \\
Household experienced drought in past year & 0.385 & 0.487 & 0.380 & 0.486 & -0.03 & (0.03) \\
Household owns a cell phone (excluding group phone) & 0.296 & 0.457 & 0.295 & 0.457 & -0.00 & (0.03) \\
Access to household or village-level cell phone & 0.763 & 0.426 & 0.798 & 0.402 & 0.04* & (0.02) \\
Respondent has used cell phone since last harvest &

In [40]:
# File path
file_path = '../manuscript/Tables/contamination_of_the_randomization.tex'

# Save the LaTeX table to a file
with open(file_path, 'w') as f:
    f.write(modified_table)

file_path

'../manuscript/Tables/contamination_of_the_randomization.tex'